In [4]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd


In [1]:
!which jupyter

/Users/dikshanasa/Desktop/Quantium/quantium-starter-repo/dash_env/bin/jupyter


In [1]:
!pip uninstall -y dash jupyter-dash
!pip install dash==2.14.2 jupyter-dash==0.4.2


Found existing installation: dash 3.0.3
Uninstalling dash-3.0.3:
  Successfully uninstalled dash-3.0.3
Found existing installation: jupyter-dash 0.4.2
Uninstalling jupyter-dash-0.4.2:
  Successfully uninstalled jupyter-dash-0.4.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 23.8 MB/s eta 0:00:000:00:0136m0:00:01:01
  Using cached jupyter_dash-0.4.2-py3-none-any.whl (23 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl (3.8 kB)

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

app = Dash(__name__)

app.layout = html.Div([
    html.H1(
        "Pink Morsels Sales Visualizer", 
        style={'textAlign': 'center', 'color': '#4CAF50', 'fontSize': 36}
    ),
    dcc.RadioItems(
        id='region-selector',
        options=[
            {'label': 'North', 'value': 'north'},
            {'label': 'East', 'value': 'east'},
            {'label': 'South', 'value': 'south'},
            {'label': 'West', 'value': 'west'},
            {'label': 'All', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'inline-block', 'padding': '10px', 'fontSize': 18},
        style={'margin': '20px', 'textAlign': 'center'}
    ),
    dcc.Graph(id='sales-graph', style={'margin': '0 auto', 'width': '80%'})
])

if __name__ == '__main__':
    app.run_server(debug=True)
